# Reto: Implementando un sistema de gestión de identidad para una Smart City

## Parte 1: Configuración del Entorno

Primero, instalemos las dependencias necesarias:

In [ ]:
# Instalar dependencias principales (LangChain moderno)
!pip install langchain langchain-openai langchain-community fastapi uvicorn pydantic pytest httpx python-dotenv radon

In [ ]:
import fastapi
import pydantic
import pytest
from langchain import __version__ as langchain_version
from langchain_openai import ChatOpenAI

print(f"FastAPI: {fastapi.__version__}")
print(f"Pydantic: {pydantic.__version__}")
print(f"LangChain: {langchain_version}")
print("✅ Entorno base configurado correctamente")

# Configurar OpenAI API Key (asegúrate de tener tu clave en .env)
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY no encontrada. Configura tu API key en archivo .env")
else:
    print("✅ OpenAI API Key configurada")

## Parte 2: Prompting Estructurado - Metodología C.R.E.A.T.E

Antes de generar código, establecemos la metodología de prompting responsable:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional


In [ ]:
class CodeOutput(BaseModel):
    code: str = Field(..., description="El código completo y funcional.")
    explanation: Optional[str] = Field(None, description="Explicación breve de las decisiones técnicas.")
    tests: str = Field(..., description="Pruebas unitarias para el código.")
    improvements: Optional[str] = Field(None, description="Sugerencias para mejorar el código.")

parser = PydanticOutputParser(pydantic_object=CodeOutput)


In [ ]:
model = ChatOpenAI(model="gpt-5", temperature=0)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
     {rol}

     CONTEXTO: {contexto}

    STACK TECNOLÓGICO RESTRINGIDO (MANDATORIO):
    Estricta adherencia a las siguientes tecnologías. No usar alternativas.
     
    Backend: {backend}
    Frontend: {frontend}
    Base de datos: {database}
    Autenticación: {auth}
    Despliegue: {deployment}
     
    PRINCIPIOS DE DESARROLLO:
    - Código limpio y modular
    - Buenas prácticas de seguridad
    - Documentación clara y concisa
    - Pruebas unitarias y de integración
    - Tests que cubran casos borde
    - Manejo de errores explícito
    - Comentarios que expliquen el "por qué", no el "qué"
    """),
    ("human", """
    OBJETIVO: {objetivo}
    RESTRICCIONES: 
     {restricciones}
    FASES DE DESARROLLO ASISTIDO POR IA
    FASE 1: Planificación y Setup (INSTRUCCIÓN INICIAL)
    {fase_1}
    FASE 2: Desarrollo Backend Core (TAREAS CRUDAS)
    {fase_2}
     FASE 3: Desarrollo Frontend Esencial (TAREAS CRUDAS)
    {fase_3}
     FASE 4: Integración y Documentación (TAREAS CRUDAS)
    {fase_4}
    FASE 5: Pruebas y Optimización (TAREAS CRUDAS)
    {fase_5}
    CONTEXTO ADICIONAL: {contexto_adicional}
    {format_instructions}
     """)
])

In [ ]:
chain = prompt | model | parser

## Parte 3: Generación del Endpoint

Usamos el prompt anterior para generar nuestro código base:

In [ ]:
import asyncio
import time
from datetime import datetime


inputs = {
    "rol": "Actúa como un Arquitecto de Soluciones de IA (AI Solution Architect) y Desarrollador Full-Stack Senior especializado en sistemas de ciudades inteligentes y tecnologías IoT",
    "contexto": "Estás encargado de guiarme a través de un reto de desarrollo rápido de 4 fases para crear un Sistema Central de Gestión de Movilidad Urbana (Smart Mobility Hub).",
    "backend": "Python con FastAPI",
    "frontend": "React con TypeScript.",
    "database": "PostgreSQL",
    "auth": "OAuth 2.0 con JWT",
    "deployment": "Enfoque en la Contenedorización para despliegue y uso de variables de ambiente (.env)",
    "objetivo": "Crear un prototipo funcional de un Sistema de Gestión de Tráfico y Movilidad de una Smart City que centralice datos de sensores IoT.",
    "restricciones": "Usar Pydantic para validaciones, incluir docstrings, manejar duplicados, máximo 30 líneas por función",
    "fase_1": """ 
        1. Análisis de Caso: Resumen del alcance, tecnologías clave y posibles retos de seguridad/escalabilidad.
        2. Arquitectura: Diagrama de alto nivel (texto/markdown) para el sistema (Backend, DB, Frontend, IoT Ingestion).
        3. Estructura Inicial: Comandos de terminal y estructura de carpetas tree para la base del proyecto backend, frontend y contenerización.
        """,
    "fase_2": """
        1. API Esencial: Código de un archivo main (Python) con inicialización completa. 
        2. Modelos y Schemas: Código Pydantic (FastAPI) para User y SensorData (ID, Lat/Lon, Tipo, Valor, Timestamp).
        3. Autenticación (Placeholder): Implementación de una función básica de login con token Oauth. 
        4. Endpoints: Código para dos rutas: /users/me (GET) y /data/iot (POST para ingesta).
        5. Cuida las dependencias entre módulos y la estructura de carpetas.
    """,
    "fase_3": """
        1. Interfaz Principal: Estructura HTML/JSX y componentes core (Login, Dashboard) propon un diseño atractivo y funcional.
        2. Conexión: Función Fetch/Axios de ejemplo para el login.
        3. Visualización: Genera el código para un componente que muestre los datos de SensorData como una Tabla o Gráfico de Barras (Usando una librería JS simple como Chart.js).
        4. Agrega estilos visuales al frontend para hacer mas atractiva la aplicacion.
        """,
    "fase_4": """
        1. Docker Setup: Archivo Dockerfile para frontend y backend funcional cuidando la estructura de carpetas.
        2. Orquestación: Archivo docker-compose.yml para ejecutar el frontend, backend y la DB tomando en cuenta las dependencias entre cada contenedor, cuida que las rutas coincidan con la estructura de carpetas.
        3. Documentación: Un README.md detallado (Setup, Endpoints, Tecnologías, Retos, Buenas practicas, Tests).
        4. Validación: Pasos de testing terminal (cURL o similar) para validar el sistema.
        5. Cuida las dependencias entre módulos y la estructura de carpetas.
    """,
    "fase_5": """
        1. Pruebas Unitarias: Código de tests (pytest) para los endpoints /users/me y /data/iot.
        2. Manejo de Errores: Código para manejar errores comunes (400, 401, 500) en el Backend.
        3. Optimización: Aplicacion de mejoras de rendimiento y seguridad (5 puntos).
        4. Revisión Final: Resumen de todo el código generado y pasos siguientes recomendados.
    """,
    "contexto_adicional": """
        Caso de Estudio Base: Desarrollaremos un sistema que monitorea sensores de tráfico y calidad del aire en 3 zonas de la ciudad para optimizar las rutas y mejorar la calidad ambiental.
        Criterios de Éxito: La solución debe ser segura, modular y demostrar la integración de datos IoT en una interfaz de usuario atractive.
        """,
    "format_instructions": parser.get_format_instructions()
}

In [ ]:
print("🚀 GENERANDO CÓDIGO CON IA...")
print(f"⏰ Inicio: {datetime.now().strftime('%H:%M:%S')}")

start_time = time.time()

result = chain.invoke(inputs)

generation_time = time.time() - start_time

In [ ]:
print("Codigo generado.")
print(result.code)

In [ ]:
print(result.tests)

In [ ]:
# ANÁLISIS AUTOMÁTICO del código generado con métricas
import ast
import re
from collections import Counter


code_to_analyze = result.code

print("📊 ANÁLISIS AUTOMÁTICO DEL CÓDIGO GENERADO")
print("="*50)

# 1. Análisis básico de estructura
lines = []
non_empty_lines = [line for line in lines if line.strip()]

print(f"📏 Estructura:")
print(f"  - Total líneas: {len(lines)}")
print(f"  - Líneas con código: {len(non_empty_lines)}")
print(f"  - Líneas comentario: {sum(1 for line in lines if line.strip().startswith('#'))}")
print(f"  - Líneas docstring: {code_to_analyze.count('\"\"\"') // 2}")

# 2. Análisis de funciones y clases
function_count = len(re.findall(r'def\s+\w+', code_to_analyze))
class_count = len(re.findall(r'class\s+\w+', code_to_analyze))
async_function_count = len(re.findall(r'async\s+def\s+\w+', code_to_analyze))

print(f"\n🏗️ Componentes:")
print(f"  - Funciones: {function_count}")
print(f"  - Funciones async: {async_function_count}")
print(f"  - Clases: {class_count}")

# 3. Análisis de imports y dependencias
imports = re.findall(r'from\s+(\w+)|import\s+(\w+)', code_to_analyze)
all_imports = [imp[0] or imp[1] for imp in imports]
import_counter = Counter(all_imports)

print(f"\n📦 Dependencias detectadas:")
for imp, count in import_counter.most_common():
    print(f"  - {imp}: {count} usos")

# 4. Análisis de patrones FastAPI
fastapi_patterns = {
    'endpoints': len(re.findall(r'@app\.(get|post|put|delete)', code_to_analyze)),
    'pydantic_models': len(re.findall(r'class\s+\w+\(BaseModel\)', code_to_analyze)),
    'field_validations': len(re.findall(r'Field\(', code_to_analyze)),
    'exception_handling': len(re.findall(r'HTTPException', code_to_analyze)),
    'response_models': len(re.findall(r'response_model=', code_to_analyze))
}

print(f"\n🚀 Patrones FastAPI:")
for pattern, count in fastapi_patterns.items():
    print(f"  - {pattern.replace('_', ' ').title()}: {count}")

# 5. Score de calidad básico
quality_score = 0
max_score = 10

# Criterios de calidad
if function_count > 0: quality_score += 2
if class_count > 0: quality_score += 2
if code_to_analyze.count('"""') >= 2: quality_score += 2  # Docstrings
if 'HTTPException' in code_to_analyze: quality_score += 1  # Error handling
if 'Field(' in code_to_analyze: quality_score += 1  # Pydantic validations
if async_function_count > 0: quality_score += 1  # Async support
if len(non_empty_lines) > 15: quality_score += 1  # Sufficient code

print(f"\n⭐ SCORE DE CALIDAD: {quality_score}/{max_score}")

if quality_score >= 8:
    print("🎉 Excelente - Código listo para producción")
elif quality_score >= 6:
    print("👍 Bueno - Algunas mejoras menores recomendadas")
elif quality_score >= 4:
    print("⚠️  Aceptable - Necesita mejoras significativas")
else:
    print("❌ Problemático - Requiere refactoring mayor")
  

In [ ]:
from langchain_core.runnables import RunnableLambda


# 1. Función de análisis de calidad
def analyze_code_quality(code: str) -> dict:
    """Analizar calidad y detectar posibles mejoras"""
    issues = []
    suggestions = []
    
    # Detectar problemas comunes
    if len(code.split('\n')) < 10:
        issues.append("Código muy corto, podría estar incompleto")
    
    if 'HTTPException' not in code:
        issues.append("Falta manejo de errores HTTP")
        suggestions.append("Añadir HTTPException para casos de error")
    
    if 'Field(' not in code and 'BaseModel' in code:
        issues.append("Modelos Pydantic sin validaciones Field")
        suggestions.append("Añadir Field() con validaciones específicas")
    
    if code.count('"""') < 2:
        issues.append("Documentación insuficiente")
        suggestions.append("Añadir docstrings a funciones y clases")
    
    function_lines = []
    in_function = False
    current_function_lines = 0
    
    for line in code.split('\n'):
        if line.strip().startswith('def ') or line.strip().startswith('async def '):
            if in_function:
                function_lines.append(current_function_lines)
            in_function = True
            current_function_lines = 1
        elif in_function:
            if line.strip() and not line.startswith('    '):
                function_lines.append(current_function_lines)
                in_function = False
                current_function_lines = 0
            else:
                current_function_lines += 1
    
    if in_function:
        function_lines.append(current_function_lines)
    
    long_functions = [lines for lines in function_lines if lines > 20]
    if long_functions:
        issues.append(f"Funciones muy largas: {len(long_functions)} funciones >20 líneas")
        suggestions.append("Refactorizar funciones largas en funciones más pequeñas")
    
    return {
        "issues": issues,
        "suggestions": suggestions,
        "quality_score": max(0, 10 - len(issues))
    }


analyze_code_quality(result.code)

In [ ]:
refinement_promt = ChatPromptTemplate.from_messages([
    ("system", """Eres un code reviewer senior experto en FastAPI y seguridad.
    
    Tu trabajo es MEJORAR código existente basándote en issues detectados automáticamente.
    
    PRINCIPIOS:
    - Mantén la funcionalidad original
    - Aplica solo mejoras específicas solicitadas
    - No reescribas completamente, solo mejora incrementalmente
    - Añade comentarios para explicar mejoras
    - Detecta vulnerabilidades de seguridad comunes
    - Prioriza mejoras que aumenten robustez y mantenibilidad
    - Elimina código muerto y dependencias innecesarias
    """),
    ("human", """
    CÓDIGO ORIGINAL:
    {codigo}
    
    ISSUES DETECTADOS:
    {issues}
    
    MEJORAS SUGERIDAS:
    {suggestions}
    
    Aplica las mejoras más importantes manteniendo la estructura original.
    Devuelve el código mejorado y explica qué cambios hiciste.
    """)
])


In [ ]:
class RefinedCodeOutput(BaseModel):
    codigo_mejorado: str = Field(description="Código con mejoras aplicadas")
    cambios_realizados: str = Field(description="Lista de cambios específicos aplicados")
    score_calidad_estimado: int = Field(ge=1, le=10, description="Score de calidad estimado tras mejoras")



In [ ]:
refinement_chain = refinement_promt | model.with_structured_output(RefinedCodeOutput)

In [ ]:
# 3. Pipeline completo de refinamiento
def refine_generated_code(code: str) -> RefinedCodeOutput:
    """Pipeline completo de análisis y refinamiento"""
    
    # Analizar calidad actual
    quality_analysis = analyze_code_quality(code)
    
    if quality_analysis["quality_score"] >= 10:
        print("✅ Código ya tiene alta calidad, refinamiento mínimo")
    
    # Aplicar refinamiento si es necesario
    refinement_input = {
        "codigo": code,
        "issues": "\n".join(f"- {issue}" for issue in quality_analysis["issues"]),
        "suggestions": "\n".join(f"- {sugg}" for sugg in quality_analysis["suggestions"])
    }
    
    if quality_analysis["issues"]:
        print(f"🔧 Aplicando {len(quality_analysis['issues'])} mejoras...")
        refined = refinement_chain.invoke(refinement_input)
        return refined
    else:
        # Sin mejoras necesarias
        return RefinedCodeOutput(
            codigo_mejorado=code,
            cambios_realizados="No se requirieron cambios - código ya óptimo",
            score_calidad_estimado=quality_analysis["quality_score"]
        )



In [ ]:
refined_result = refine_generated_code(result.code)

In [ ]:
print(refined_result.codigo_mejorado)

In [ ]:
print(refined_result.cambios_realizados)